In [1]:
from datetime import datetime
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import gzip

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

In [3]:
train = pd.read_csv('../data/train.csv.gz', parse_dates=True,
                    )

In [4]:
label_encoder = LabelEncoder()

def get_dates(d):
    dt = datetime.strptime(d, '%Y-%m-%d %H:%M:%S')
    return dt.hour, dt.month, dt.year

def get_features(data):
    df = pd.DataFrame()
    df['Hour'], df['Month'], df['Year'] = zip(*data['Dates'].apply(get_dates))
    df = df.join(pd.get_dummies(data['PdDistrict'], prefix='District'))
    df = df.join(pd.get_dummies(data['DayOfWeek'], prefix='Day'))
    labels = None
    if 'Category' in data:
        labels = label_encoder.fit_transform(data['Category'])
    return df, labels

In [5]:
features, labels = get_features(train)
scaler = StandardScaler()
scaler.fit(features)
features = scaler.transform(features)

In [6]:
# Esto consume mucha RAM, probar en una maquina mas potente con mas n_estimators
rf_model = RandomForestClassifier(n_estimators=10)
rf_model.fit(features, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
test = pd.read_csv('../data/test.csv.gz', parse_dates=True,
                   )

In [8]:
test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [9]:
test_features, _ = get_features(test)

scaler.fit(test_features)
test_features = scaler.transform(test_features)

In [10]:
proba = rf_model.predict_proba(test_features)
prediction = pd.DataFrame(proba, columns=label_encoder.classes_)

In [11]:
with gzip.GzipFile('rf.csv.gz', mode='w') as f:
    prediction.to_csv(f, index_label="Id", na_rep="0")